In [44]:

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [45]:
#load the pdf   from folder
loader = PyPDFDirectoryLoader('./us_census') 
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
final_documents[0]

#https://python.langchain.com/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census/acsbr-015.pdf', 'page': 0})

In [26]:
len(final_documents)

316

In [27]:
huggingface_embeddings =HuggingFaceBgeEmbeddings(
    # https://huggingface.co/BAAI/bge-small-en-v1.5#using-langchain  used
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [28]:

import numpy as np

print(np.array(huggingface_embeddings.embed_query(
    final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(
    final_documents[0].page_content)).shape)

[-8.46568421e-02 -1.19099263e-02 -3.37892585e-02  2.94559319e-02
  5.19159697e-02  5.73839881e-02 -4.10017520e-02  2.74268147e-02
 -1.05128184e-01 -1.58055648e-02  7.94858560e-02  5.64318784e-02
 -1.31765548e-02 -3.41544263e-02  5.81604103e-03  4.72547784e-02
 -1.30746746e-02  3.12991883e-03 -3.44225839e-02  3.08406446e-02
 -4.09085974e-02  3.52737755e-02 -2.43761428e-02 -4.35831398e-02
  2.41503436e-02  1.31986672e-02 -4.84449416e-03  1.92347690e-02
 -5.43912612e-02 -1.42735034e-01  5.15532494e-03  2.93115471e-02
 -5.60810938e-02 -8.53536651e-03  3.14141065e-02  2.76736412e-02
 -2.06188299e-02  8.24231729e-02  4.15425301e-02  5.79655133e-02
 -3.71587127e-02  6.26156712e-03 -2.41390411e-02 -5.61796688e-03
 -2.51715500e-02  5.04968967e-03 -2.52800845e-02 -2.91943154e-03
 -8.24043527e-03 -5.69604523e-02  2.30822805e-02 -5.54220937e-03
  5.11555523e-02  6.09937645e-02  6.49766102e-02 -5.38513996e-02
  2.19109785e-02 -2.54194643e-02 -4.49223518e-02  4.22458798e-02
  4.75251935e-02  7.23215

In [29]:
# VectorStore Creation
vectorstore = FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [30]:
# Query using Similarity Search
query = "WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments = vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [31]:

#https://python.langchain.com/docs/how_to/vectorstore_retriever/
# A vector store retriever is a retriever that uses a vector store to retrieve documents. 
# It is a lightweight wrapper around the vector store class to make it conform to the retriever interface. 
# It uses the search methods implemented by a vector store, like similarity search and MMR, to query the texts in the vector store.

retriever = vectorstore.as_retriever(  # instantiate a retriever:
    search_type="similarity", search_kwargs={"k": 3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x727ea08fad10> search_kwargs={'k': 3}


In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN") 
# https://huggingface.co/docs/hub/main/index


In [33]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500}

)
query = "What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is a benefit that is provided by the employer to the employees. The health insurance coverage is a benefit that is provided by the employer to the employees. The health insurance coverage is a benefit that is provided by the employer to the employees. The health insurance coverage is a benefit that is provided by the employer to the employees. The health insurance coverage is a benefit that is provided by the employer to the employees. The health insurance coverage is a benefit that is provided by the employer'

In [ ]:
# # Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
# )

# llm = hf
# llm.invoke(query)

In [34]:
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [35]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [42]:
#
retreivalQA = RetrievalQA.from_chain_type(
         llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)
    


In [43]:
# # Call the QA chain with our query.
# result = retreivalQA.invoke({"query": query})
# print(result['result'])

chain.invoke({"query": query})

KeyError: 'input'